In [1]:
# !pip install -q -r requirements.txt')
# !pip install -q stanza
from adjective_reading import *

In [2]:
next(STASH_NLP.items())

((None,
  'Although my reading in Latin Christianity has in fact led to very different conclusions from his, that same read- ing has led me to share Robertson’s interest in how medieval people read and how we read their reading now.'),
 b'\x80\x04\x95\xa7\x13\x00\x00\x00\x00\x00\x00\x8c\xdfAlthough my reading in Latin Christianity has in fact led to very different conclusions from his, that same read- ing has led me to share Robertson\xe2\x80\x99s interest in how medieval people read and how we read their reading now.\x94]\x94]\x94(}\x94(\x8c\x02id\x94K\x01\x8c\x04text\x94\x8c\x08Although\x94\x8c\x05lemma\x94\x8c\x08although\x94\x8c\x04upos\x94\x8c\x05SCONJ\x94\x8c\x04xpos\x94\x8c\x02IN\x94\x8c\x04head\x94K\n\x8c\x06deprel\x94\x8c\x04mark\x94\x8c\nstart_char\x94K\x00\x8c\x08end_char\x94K\x08\x8c\x03ner\x94\x8c\x01O\x94\x8c\tmulti_ner\x94h\x13\x85\x94u}\x94(h\x04K\x02h\x05\x8c\x02my\x94h\x07\x8c\x02my\x94h\t\x8c\x04PRON\x94h\x0b\x8c\x04PRP$\x94\x8c\x05feats\x94\x8c3Case=Gen|Number=Sing|

In [29]:
def iter_parsed_sents(limit=None,**kwargs):
    import stanza
    avail = len(STASH_NLP)
    total = limit if limit and limit < avail else avail
    for i, (sentstr, sentdoc) in enumerate(STASH_NLP.items(), 1):
        if i > total:
            break
        yield sentstr, stanza.Document.from_serialized(sentdoc)

In [31]:
sentstr,sentdoc=next(iter_parsed_sents(limit=1))
sentdoc.num_tokens, sentstr

(50,
 '\ue062e contexts part nicely sums up different directions in the reception and cri- ticism of Popov’s texts, in a way that clearly demonstrates both the difficulties in attaching labels to Popov’s writings and the need to combine general assessments of his writings with careful readings of specific texts.')

In [32]:

def get_parsed_sent_stats(sentdoc, keyword = None):
    tok_ld = sentdoc.to_dict()
    tok_id2obj = {}
    tok_id2d = {}
    tok_id2deprel = {}
    tok_id2head = {}
    tok_dd = {}

    for sent in sentdoc.sentences:
        for tok in sent.tokens:
            tok_d = tok.to_dict()[0]
            tok_id = tok_d['id']
            tok_id2d[tok_id] = tok_d
            tok_id2obj[tok_id] = tok

    out_ld = []
    for tok_id, tok_d in tok_id2d.items():
        tok_text = tok_d['text']
        if not keyword or tok_text.lower() == keyword.lower():
            tok_head_id = tok_d.get('head',0)
            tok_head_d = tok_id2d.get(tok_head_id,{})
            tok_id2head[tok_id] = tok_head_id
            ttext = tok_d.get("text","[text]")
            thead = tok_head_text = tok_head_d.get("text","[head]").lower()
            tdeprel = tok_id2deprel[tok_id] = tok_d.get('deprel',"[rel]")

            out_d = {
                'id': tok_id,
                'text': ttext,
                'head': thead,
                'deprel': tdeprel,
                'sentstr': sentstr,
                'sent': detokenize_and_uppercase(sentdoc, token_ids=[tok_id,tok_head_id])
            }
            out_ld.append(out_d)
    return out_ld


In [35]:
out_ld = []
for sentstr, sentdoc in iter_parsed_sents(limit=None):
    out_ld.extend(get_parsed_sent_stats(sentdoc, keyword="reading"))
out_df = pd.DataFrame(out_ld)
out_df


,id,text,head,deprel,sentstr,sent
0,2,reading,[head],root,The reading adopted in vv.,The READING adopted in vv.
1,25,reading,travel,nmod,"Men did thus escape, in artistic activity, in ...","Men did thus escape, in artistic activity, in ..."
2,14,reading,practice,nmod,"But in addition to all these, rhetoric taught ...","But in addition to all these, rhetoric taught ..."
3,14,reading,writing,conj,Stanton is right: we must always be making con...,Stanton is right: we must always be making con...
4,18,reading,difficult,conj,It is difficult to take all of this seriously-...,It is DIFFICULT to take all of this seriously-...
...,...,...,...,...,...,...
4036,2,reading,change,nsubj,"This reading, if correct, would not change our...","This READING, if correct, would not CHANGE our..."
4037,37,reading,speaking,conj,"In the biographies of Guthlac, the violence of...","In the biographies of Guthlac, the violence of..."
4038,14,reading,agree,ccomp,Rabbi Yehoshua gives this verse what we would ...,Rabbi Yehoshua gives this verse what we would ...
4039,8,reading,explanation,conj,"In fact, no real explanation or reading of ""Vo...","In fact, no real EXPLANATION or READING of ""Vo..."


In [23]:
row = out_df.sample(1).iloc[0]
print(row.text)
print(row.deprel)
print(row['head'])

pprint(row.sent)


reading
obj
show
('G. C. M. S. Women Writers of the Nineteenth Century, by Marjory A. Bald '
 '(Cambridge University Press), is a series of studies of Jane Austen, the '
 'Brontes, Mrs Gaskell, George Eliot, Mrs Browning and Christina Rossetti, '
 'which SHOW careful READING, power of entering into the minds of their '
 'subjects, and sound criticism.')


In [47]:
def get_reading_tok(sentdoc):
    for sentstr, sentdoc in iter_parsed_sents(limit=1):
        for sent in sentdoc.sentences:
            for tok in sent.tokens:
                if tok.text.lower() == 'reading':
                    return tok

In [48]:
sentdoc = get_nlp_doc("Close, nuanced reading stresses something important.")

In [49]:
readtok = get_reading_tok(sentdoc)

In [46]:
readtok

[
  {
    "id": 2,
    "text": "reading",
    "lemma": "reading",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "head": 0,
    "deprel": "root",
    "start_char": 4,
    "end_char": 11,
    "ner": "O",
    "multi_ner": [
      "O"
    ]
  }
]

In [43]:
pprint(tok.to_dict())

[{'deprel': 'det',
  'end_char': 3,
  'feats': 'Definite=Def|PronType=Art',
  'head': 2,
  'id': 1,
  'lemma': 'the',
  'multi_ner': ('O',),
  'ner': 'O',
  'start_char': 0,
  'text': 'The',
  'upos': 'DET',
  'xpos': 'DT'}]
